In [1]:
import tensorflow as tf
import keras as k
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from sklearn.utils import shuffle
import os, argparse, random, pickle, cv2, fnmatch
from os.path import join

%matplotlib inline

Using TensorFlow backend.


# Labels
## Create a list of the paths of the files that contain the labels

In [2]:
Labels = []
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename


for filename in find_files('/data/home/maj/notebooks/w210-final-project/MAJ/Frame_Labels/PSPI/', '*.txt'):
    Labels.append(filename)

Labels_df = pd.DataFrame(data=Labels)

Labels_df.shape


(48398, 1)

# Images
## Create a list of the paths of the files that contain the Images

In [3]:
images_path = []
def find_files(directory, pattern):
   for root, dirs, files in os.walk(directory):
       for basename in files:
           if fnmatch.fnmatch(basename, pattern):
               filename = os.path.join(root, basename)
               yield filename


for filename in find_files('/data/home/maj/notebooks/w210-final-project/MAJ', '*.png'):
   images_path.append(filename)

images_path_df = pd.DataFrame(data=images_path)

images_path_df.shape

(48398, 1)

# Creating a list of images
## This is the part I don't understand.  We need to simplify this, or package it in a function

In [4]:
images = []
sequences = []
image_sequences = []

pspi_scores = []
opr_scores = []

aff_scores = []
sen_scores = []
vas_scores = []

for subject_id in os.listdir('Images'):
    if subject_id != '.DS_Store':
        for sequence_id in os.listdir(join('Images', subject_id)):
            if sequence_id != '.DS_Store':
                sequences.append(sequence_id)
                for image in os.listdir(join('Images', subject_id, sequence_id)):
                    if image != '.DS_Store':
                        images.append(image.split('.')[0])
                        image_sequences.append(sequence_id)
                for pspi_file in os.listdir(join('Frame_Labels', 'PSPI', subject_id, sequence_id)):
                    with open(join('Frame_Labels', 'PSPI', subject_id, sequence_id, pspi_file), 'r') as f_in:
                        pspi_scores.append(float(f_in.read().strip()))
        for aff_file in os.listdir(join('Sequence_Labels', 'AFF', subject_id)):
            with open(join('Sequence_Labels', 'AFF', subject_id, aff_file), 'r') as f_in:
                aff_scores.append(float(f_in.read().strip()))
        for opr_file in os.listdir(join('Sequence_Labels', 'OPR', subject_id)):
            with open(join('Sequence_Labels', 'OPR', subject_id, opr_file), 'r') as f_in:
                opr_scores.append(float(f_in.read().strip()))
        for sen_file in os.listdir(join('Sequence_Labels', 'SEN', subject_id)):
            with open(join('Sequence_Labels', 'SEN', subject_id, sen_file), 'r') as f_in:
                sen_scores.append(float(f_in.read().strip()))
        for vas_file in os.listdir(join('Sequence_Labels', 'VAS', subject_id)):
            with open(join('Sequence_Labels', 'VAS', subject_id, vas_file), 'r') as f_in:
                vas_scores.append(float(f_in.read().strip()))

image_data = {'image': images,
              'image_sequence': image_sequences,
              'pspi_score': pspi_scores}

sequence_data = {'sequence': sequences, 
                 'aff_score': aff_scores, 
                 'opr_score': opr_scores, 
                 'sen_score': sen_scores,
                 'vas_score': vas_scores}

image_df = pd.DataFrame(data=image_data)
sequence_df = pd.DataFrame(data=sequence_data)


In [5]:
print ('Examples of image identifiers:\n{}\n'.format(images[:10]))
print ('Examples of sequence identifiers:\n{}\n'.format(sequences[:10]))
print ('Examples of PSPI scores:\n{}\n'.format(pspi_scores[:10]))
print ('Examples of AFF scores:\n{}\n'.format(aff_scores[:10]))
print ('Examples of OPR scores:\n{}\n'.format(opr_scores[:10]))
print ('Examples of SEN scores:\n{}\n'.format(sen_scores[:10]))
print ('Examples of VAS scores:\n{}\n'.format(vas_scores[:10]))

Examples of image identifiers:
['fn059t2afunaff048', 'fn059t2afunaff405', 'fn059t2afunaff119', 'fn059t2afunaff163', 'fn059t2afunaff003', 'fn059t2afunaff005', 'fn059t2afunaff022', 'fn059t2afunaff176', 'fn059t2afunaff287', 'fn059t2afunaff257']

Examples of sequence identifiers:
['fn059t2afunaff', 'fn059t2aiaff', 'jh043t2afunaff', 'jh043t2afaff', 'jh043t2aeunaff', 'jh043t2aiunaff', 'jh043t1afaff', 'jh043t2aeaff', 'jh043t2aaunaff', 'jh043t1aeaff']

Examples of PSPI scores:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Examples of AFF scores:
[7.0, 1.0, 4.0, 1.0, 1.0, 4.0, 1.0, 1.0, 13.0, 5.0]

Examples of OPR scores:
[4.0, 3.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0]

Examples of SEN scores:
[8.0, 1.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.0, 12.0, 5.0]

Examples of VAS scores:
[6.0, 1.0, 2.0, 0.0, 0.0, 4.0, 0.0, 1.0, 6.0, 3.0]



In [6]:
pain_data = image_df[['image','pspi_score']]

In [7]:
#extracting the .png file name from the first dataset and joining with the image & PSPI score dataset
images_path_df

#renaming dataframe column to image path
images_path_df.columns = ['image_path']

# new temp data frame with split value columns 
temp = images_path_df["image_path"].str.split("/", n = 10, expand = True) 
temp.columns = ['zero','one','two','three','four','five','six','seven', 'eight','nine', 'ten']


# Create new temp dataframe that has image name without .png to later do a merge
temp2 = temp["ten"].str.split(".", n = 1, expand = True) 
temp2.columns = ['image','ext']
temp2.drop(columns =["ext"], inplace = True) 

# print(temp2[:1])
# print(images_path_df[:1])

df_columns = pd.concat([images_path_df, temp2], axis =1)
print(df_columns[:1])

# #checking the size
# print('manip:', df_columns.shape)
# print('original:', pain_data.shape)

# #checking the header and first row for each data frame
# print(df_columns[:1])
# print(pain_data[:1])

#doing a merge on the .png value to ensure that the path will match the PSPI value
merged_inner = pd.merge(left=pain_data,right=df_columns, left_on='image', right_on='image')
# print('merged table: ', merged_inner.shape)
# print('New table header: ',merged_inner[:1])

#reordering columns
merged_inner= merged_inner [['image_path', 'image', 'pspi_score']]
merged_inner[:1]

#Final Dataset for pain dataset to by used in CNN
pain_data = merged_inner
print('\nFinal Dataset ready for CNN')

pain_data[:1]




                                          image_path              image
0  /data/home/maj/notebooks/w210-final-project/MA...  fn059t2afunaff048

Final Dataset ready for CNN


,image_path,image,pspi_score
0,/data/home/maj/notebooks/w210-final-project/MA...,fn059t2afunaff048,0.0


# Load & Save to Disk

In [8]:
# Shuffle Dataset
random.seed(42)
pain_data = shuffle(pain_data)

# initialize the data and labels


# initialize the data and labels
print("[INFO] loading images...")
# data_pic = []
imagelist = [] #markedbymark
pic_path = []
# labels = pain_data['pspi_score']
labellist = pain_data['pspi_score'].tolist() #markedbymark

# loop over the input images
imagePath = list(pain_data['image_path'])

for imagePath in imagePath:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (320, 240)) # resize
    imagelist.append(image)
    pic_path.append(imagePath)
    
print("[INFO] Pictures loaded")

finalArray = np.array(imagelist)
finalLabels = np.array(labellist)
print('[INFO] Array of data: {}, array of labels: {}'.format(finalArray.shape, finalLabels.shape))
print('[INFO] Saving ...')
np.save('ArrayOfImages', finalArray)
np.save('Labels', finalLabels)
print('[INFO] Datasets Saved')

[INFO] loading images...
[INFO] Pictures loaded
[INFO] Array of data: (48398, 240, 320, 3), array of labels: (48398,)
[INFO] Saving ...
[INFO] Datasets Saved


# Splitting the Data

In [11]:
test_ratio = 0.3
split_location = int(finalArray.shape[0]*(1-test_ratio))
train_data = finalArray[:split_location]
test_data = finalArray[split_location:]

train_labels = finalLabels[:split_location]
test_labels = finalLabels[split_location:]

print('Train Data\t: {}\nTrain Labels\t: {}\nTest Data\t: {}\nTest Labels\t: {}'
      .format(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape))

Train Data	: (33878, 240, 320, 3)
Train Labels	: (33878,)
Test Data	: (14520, 240, 320, 3)
Test Labels	: (14520,)


# Tensorflow Model

In [30]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(16, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.train.AdamOptimizer(), loss = 'sparse_categorical_crossentropy')

# markedbymark
model.fit(train_data, train_labels, epochs=1, batch_size=128)

33878/33878 [==============================] - 33s 988us/sample - loss: 16.0919


In [24]:
model.evaluate(test_data, test_labels)

14520/14520 [==============================] - 13s 914us/sample - loss: 2.8384


2.8384410419411568